In [2]:
import pandas as pd

In [3]:
df_itemDaten = pd.read_parquet('merged_data_with_pick_costs.parquet')
# sortiere random nach WERKS das df
df_itemDaten = df_itemDaten.sample(frac=1, random_state=42).reset_index(drop=True)
df_itemDaten.head(400).to_csv('merged_data_with_pick_costs.csv', index=False)

In [4]:
df_itemDaten.head(400)

,LFART_Types,LFART_TaO_Delivery_Types_for_3PL_Deutsch,WERKS,LGORT,LFDAT,VBELN,POSNR,PSTYV,ERNAM_x,MATNR,...,KUNNR,NAME1,ORT01,PSTLZ,Ship-To,Plnt,Pickkosten,Monat,Jahr,Monat_Jahr
0,Outbound,ZLF (Standardlieferung),DE58,A101,2024-04-22,3202718595,160,ZTAN,98999033,10192269,...,50033745,Lekkerland SE,Elz,65604,None,DE58,0.469756,4.0,2024.0,04.2024
1,Outbound,ZLF (Standardlieferung),DE54,A101,2024-06-10,3202971318,180,ZTAN,98999033,10198467,...,50034809,Lekkerland SE,Braak,22145,None,DE54,1.409268,6.0,2024.0,06.2024
2,Outbound,ZLF (Standardlieferung),DE19,A104,2025-01-08,3203224088,30,ZTN1,_DE_CPS_BTCH,10194411,...,51809254,BAT G - B2C E-Com. GLO shop,Hamburg,20354,None,DE19,0.089322,1.0,2025.0,01.2025
3,Outbound,ZLF (Standardlieferung),DE19,A104,2024-09-02,3203086471,50,ZTA1,_DE_CPS_BTCH,10191249,...,51809254,BAT G - B2C E-Com. GLO shop,Hamburg,20354,None,DE19,0.679850,9.0,2024.0,09.2024
4,Outbound,ZLF (Standardlieferung),DE19,A104,2024-03-01,3202751597,10,ZTA1,_DE_CPS_BTCH,10180348,...,51735171,BAT G - B2C E-Com. VUSE shop,Hamburg,20354,None,DE19,0.089322,3.0,2024.0,03.2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,nan,nan,DE58,nan,2024-07-26,3203043473,30,ZKFE,_DE_CPS_BTCH,10190627,...,56092060,BAT G - Barverkauf Agentur POOL,Hamburg,20354,None,DE58,0.000000,7.0,2024.0,07.2024
396,Outbound,ZLF (Standardlieferung),DE19,A104,2024-09-24,3203111229,10,ZTA1,_DE_CPS_BTCH,10171629,...,51735171,BAT G - B2C E-Com. VUSE shop,Hamburg,20354,None,DE19,0.178643,9.0,2024.0,09.2024
397,Outbound,ZLF (Standardlieferung),DE19,A104,2024-05-13,3202908027,10,ZTA1,_DE_CPS_BTCH,10180347,...,51735171,BAT G - B2C E-Com. VUSE shop,Hamburg,20354,None,DE19,2.719401,5.0,2024.0,05.2024
398,Outbound,ZLF (Standardlieferung),DE19,A104,2024-01-05,3202686950,10,ZTA1,_DE_CPS_BTCH,10180345,...,51735171,BAT G - B2C E-Com. VUSE shop,Hamburg,20354,None,DE19,0.089322,1.0,2024.0,01.2024
